In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 832.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [9]:
from ultralytics import YOLO
import os

In [4]:
# Download and unzip the dataset to /content/datasets/coco128/
!mkdir -p /content/datasets
%cd /content/datasets
!curl -L "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip" -o coco128.zip
!unzip -q coco128.zip

/content/datasets
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6819k  100 6819k    0     0  6950k      0 --:--:-- --:--:-- --:--:-- 6950k


In [10]:
data_yaml = """
path: /content/datasets/coco128
train: images/train
val: images/val

names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
 10: fire hydrant
 11: stop sign
 12: parking meter
 13: bench
 14: bird
 15: cat
 16: dog
 17: horse
 18: sheep
 19: cow
 20: elephant
 21: bear
 22: zebra
 23: giraffe
 24: backpack
 25: umbrella
 26: handbag
 27: tie
 28: suitcase
 29: frisbee
 30: skis
 31: snowboard
 32: sports ball
 33: kite
 34: baseball bat
 35: baseball glove
 36: skateboard
 37: surfboard
 38: tennis racket
 39: bottle
 40: wine glass
 41: cup
 42: fork
 43: knife
 44: spoon
 45: bowl
 46: banana
 47: apple
 48: sandwich
 49: orange
 50: broccoli
 51: carrot
 52: hot dog
 53: pizza
 54: donut
 55: cake
 56: chair
 57: couch
 58: potted plant
 59: bed
 60: dining table
 61: toilet
 62: tv
 63: laptop
 64: mouse
 65: remote
 66: keyboard
 67: cell phone
 68: microwave
 69: oven
 70: toaster
 71: sink
 72: refrigerator
 73: book
 74: clock
 75: vase
 76: scissors
 77: teddy bear
 78: hair drier
 79: toothbrush
"""
with open("/content/coco128_full.yaml", "w") as f:
    f.write(data_yaml)


In [12]:
import os, shutil, random

# Paths based on your dataset structure
base_dir = "/content/datasets/coco128"
img_dir = os.path.join(base_dir, "images/train2017")
lbl_dir = os.path.join(base_dir, "labels/train2017")  # ✅ Your actual label path

# Output folders
train_img_out = os.path.join(base_dir, "images/train")
val_img_out = os.path.join(base_dir, "images/val")
train_lbl_out = os.path.join(base_dir, "labels/train")
val_lbl_out = os.path.join(base_dir, "labels/val")

os.makedirs(train_img_out, exist_ok=True)
os.makedirs(val_img_out, exist_ok=True)
os.makedirs(train_lbl_out, exist_ok=True)
os.makedirs(val_lbl_out, exist_ok=True)

# Only include images with matching label files
all_images = [
    f for f in os.listdir(img_dir)
    if f.endswith('.jpg') and os.path.exists(os.path.join(lbl_dir, f.replace('.jpg', '.txt')))
]

random.shuffle(all_images)
train_imgs = all_images[:100]
val_imgs = all_images[100:]

# Copy matching image-label pairs
def split_copy(image_list, img_out, lbl_out):
    for img_file in image_list:
        lbl_file = img_file.replace('.jpg', '.txt')
        shutil.copy(os.path.join(img_dir, img_file), os.path.join(img_out, img_file))
        shutil.copy(os.path.join(lbl_dir, lbl_file), os.path.join(lbl_out, lbl_file))

split_copy(train_imgs, train_img_out, train_lbl_out)
split_copy(val_imgs, val_img_out, val_lbl_out)

print(f"✅ Dataset split complete: {len(train_imgs)} train / {len(val_imgs)} val")


✅ Dataset split complete: 100 train / 26 val


In [17]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Or yolov8s.pt etc.
model.train(data="/content/coco128_10class.yaml", epochs=20, imgsz=640, batch=8)


Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/coco128_10class.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tr

train: Scanning /content/datasets/coco128/labels/train... 114 images, 1 backgrounds, 0 corrupt: 100%|██████████| 115/115 [00:00<00:00, 2090.53it/s]

train: New cache created: /content/datasets/coco128/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1052.4±428.5 MB/s, size: 52.3 KB)



val: Scanning /content/datasets/coco128/labels/val... 26 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00<00:00, 2309.98it/s]

val: New cache created: /content/datasets/coco128/labels/val.cache
Plotting labels to runs/detect/train6/labels.jpg... 


...

                   all         26        219      0.701      0.584      0.684      0.503



20 epochs completed in 0.595 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.5MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.22s/it]


                   all         26        219      0.708      0.571      0.684      0.506
                person         13         48      0.821      0.625      0.736      0.537
               bicycle          1          2      0.443        0.5      0.499      0.399
                   car          3         15      0.466      0.267      0.354      0.151
              airplane          2          2      0.767          1      0.995      0.946
                   bus          2          3      0.627      0.333      0.357      0.357
                 truck          2          4        0.5       0.25      0.262      0.226
         traffic light          1          1          1          0          0          0
             stop sign          1          1      0.625          1      0.995      0.697
                 bench          2          6      0.808      0.706      0.927      0.652
                   dog          2          2          1      0.871      0.995      0.748
               giraff

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  4,  5,  7,  9, 11, 13, 16, 23, 25, 26, 27, 28, 29, 32, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 50, 51, 53, 55, 56, 57, 58, 59, 60, 61, 65, 67, 71, 72, 73, 74, 77, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7da29a5f0610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035

In [19]:
# Validate your trained model
model = YOLO("/content/datasets/runs/detect/train6/weights/best.pt")  # adjust path if different
metrics = model.val()


Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 889.1±354.7 MB/s, size: 38.7 KB)


val: Scanning /content/datasets/coco128/labels/val.cache... 26 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:07<00:00,  3.50s/it]


                   all         26        219      0.708      0.571      0.684      0.506
                person         13         48      0.821      0.625      0.736      0.537
               bicycle          1          2      0.443        0.5      0.499      0.399
                   car          3         15      0.466      0.267      0.354      0.151
              airplane          2          2      0.767          1      0.995      0.946
                   bus          2          3      0.627      0.333      0.357      0.357
                 truck          2          4        0.5       0.25      0.262      0.226
         traffic light          1          1          1          0          0          0
             stop sign          1          1      0.625          1      0.995      0.697
                 bench          2          6      0.808      0.706      0.927      0.652
                   dog          2          2          1      0.871      0.995      0.748
               giraff

In [8]:

# Import and load model
from ultralytics import YOLO
from google.colab import files

# Upload image
uploaded = files.upload()

# Load YOLOv8 model (nano version or any custom .pt file you have)
model = YOLO('yolov8n.pt')  # or your trained model e.g. 'runs/detect/train/weights/best.pt'

# Detect on the uploaded image
for image_name in uploaded.keys():
    results = model.predict(source=image_name, conf=0.25, show=True)


In [22]:
model = YOLO("/content/datasets/runs/detect/train6/weights/best.pt")  # Ensure this matches your trained weights

# 🔹 Evaluate on validation dataset
metrics = model.val(data="/content/coco128_10class.yaml", split="val")

# 🔹 Print results
print("\n📊 Evaluation Metrics:")
print(f"📌 Precision: {metrics.box.map50:.4f}")
print(f"📌 Recall: {metrics.box.map75:.4f}")
print(f"📌 mAP@50: {metrics.box.map50:.4f}")
print(f"📌 mAP@50-95: {metrics.box.map:.4f}")



Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1131.3±428.5 MB/s, size: 45.8 KB)


val: Scanning /content/datasets/coco128/labels/val.cache... 26 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:07<00:00,  3.65s/it]


                   all         26        219      0.708      0.571      0.684      0.506
                person         13         48      0.821      0.625      0.736      0.537
               bicycle          1          2      0.443        0.5      0.499      0.399
                   car          3         15      0.466      0.267      0.354      0.151
              airplane          2          2      0.767          1      0.995      0.946
                   bus          2          3      0.627      0.333      0.357      0.357
                 truck          2          4        0.5       0.25      0.262      0.226
         traffic light          1          1          1          0          0          0
             stop sign          1          1      0.625          1      0.995      0.697
                 bench          2          6      0.808      0.706      0.927      0.652
                   dog          2          2          1      0.871      0.995      0.748
               giraff

In [ ]:
from google.colab import drive
drive.mount('/content/drive')